In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

### Buradaki kritik husus, calistigin directory'nin parentinda `sentence_wise_email` folderini onun icinde `module` folderini ve en icte de `module_useT` folderini create etmek

In [3]:
!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC ../sentence_wise_email/module/module_useT

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0x ./
x ./tfhub_module.pb
x ./variables/
 97  745M   97  727M    0     0  12.4M      0  0:00:59  0:00:58  0:00:01 18.9M8M      0  0:00:58  0:00:38  0:00:20 16.6M9M      0  0:00:57  0:00:39  0:00:18 16.5M
x ./variables/variables.index
x ./assets/
100  745M  100  745M    0     0  12.5M      0  0:00:59  0:00:59 --:--:-- 20.1M



In [4]:
embed = hub.Module("../sentence_wise_email/module/module_useT")

In [8]:
# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))
for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join((str(x) for x in        message_embedding[:3]))
        print("Embedding[{},...]\n".
                   format(message_embedding_snippet))

[None, None]

Message: Elephant
Embedding size: 512
Embedding[0.044984735548496246, -0.05743393301963806, 0.0022114701569080353,...]

Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding[0.05568017438054085, -0.009607937186956406, 0.006246284581720829,...]

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding[0.03874938562512398, 0.0765201598405838, -0.0007945825927890837,...]



In [9]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_useT('../sentence_wise_email/module/module_useT')
messages = [
    "we are sorry for the inconvenience",
    "we are sorry for the delay",
    "we regret for your inconvenience",
    "we don't deliver to baner region in pune",
    "we will get you the best possible rate"
]
embed_fn(messages)

array([[ 0.00021282, -0.04443965,  0.03926145, ...,  0.0483016 ,
        -0.11913303,  0.03370819],
       [ 0.01778569, -0.0519203 , -0.00792321, ..., -0.01799901,
        -0.09819184,  0.06020728],
       [-0.00748424, -0.0240158 ,  0.01762745, ...,  0.09334016,
        -0.11837555,  0.00603596],
       [-0.03505319, -0.01932572, -0.0324861 , ...,  0.00356431,
        -0.08239801,  0.03887842],
       [-0.05111802, -0.0309066 ,  0.03542012, ..., -0.01343899,
        -0.10434885, -0.03150062]], dtype=float32)

In [10]:
encoding_matrix = embed_fn(messages)
np.inner(encoding_matrix, encoding_matrix)

array([[0.9999999 , 0.87426376, 0.80048907, 0.2380786 , 0.4646979 ],
       [0.87426376, 1.0000002 , 0.60501504, 0.25081366, 0.4493389 ],
       [0.80048907, 0.60501504, 0.9999998 , 0.1784874 , 0.41954657],
       [0.2380786 , 0.25081366, 0.1784874 , 0.99999976, 0.24955791],
       [0.4646979 , 0.4493389 , 0.41954657, 0.24955791, 1.0000002 ]],
      dtype=float32)

In [11]:
#It takes similarity matrix (generated from sentence encoder) as input and gives index of redundant statements
def redundant_sent_idx(sim_matrix):
    dup_idx = [] 
    for i in range(sim_matrix.shape[0]):
        if i not in dup_idx:
            tmp = [t+i+1 for t in list(np.where( sim_matrix[i][i+1:] > 0.8 )[0])]
            dup_idx.extend(tmp)
    return dup_idx
#indexes of duplicate statements.
dup_indexes  = redundant_sent_idx(np.inner(encoding_matrix,
                                           encoding_matrix))
unique_messages = np.delete(np.array(messages), dup_indexes)

In [13]:
np.array(['we are sorry for the inconvenience',
       "we don't deliver to baner region in pune",
       'we will get you the best possible rate'], dtype='<U40')

array(['we are sorry for the inconvenience',
       "we don't deliver to baner region in pune",
       'we will get you the best possible rate'], dtype='<U40')

In [15]:
greets = ["What's up?",
 'It is a pleasure to meet you.',
 'How do you do?',
 'Top of the morning to you!',
 'Hi',
 'How are you doing?',
 'Hello',
 'Greetings!',
 'Hi, How is it going?',
 'Hi, nice to meet you.',
 'Nice to meet you.']
greet_matrix = embed_fn(greets)
test_text = "I will remove my car?"
test_embed = embed_fn([test_text])
np.inner(test_embed, greet_matrix)
sim_matrix  = np.inner(test_embed, greet_matrix)
if sim_matrix.max() > 0.8:
    print("it is a greetings")
else:
    print("it is not a greetings")

array([[0.16033182, 0.24214168, 0.30527407, 0.321598  , 0.27426553,
        0.07302819, 0.30970466, 0.3245443 , 0.0890827 , 0.29725873,
        0.2704302 ]], dtype=float32)

it is not a greetings
